In [0]:
 flight_schema_ddl = """FL_DATE STRING, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

In [0]:
flight_time_raw_df = spark.read \
                        .format("json") \
                        .schema(flight_schema_ddl) \
                        .option("mode", "FAILFAST") \
                        .option("dateFormat", "M/d/y") \
                        .load("/FileStore/tables/flight-time.json")

In [0]:
from pyspark.sql.functions import to_date, expr
flight_time_df = flight_time_raw_df \
                        .withColumn("FL_DATE", to_date("FL_DATE", "M/d/y")) \
                        .withColumn("CANCELLED", expr("if(CANCELLED==1, true, false)"))

In [0]:
%fs rm -r /user/hive/warehouse/

In [0]:
flight_time_df.write \
            .format("parquet") \
            .mode("overwrite") \
            .saveAsTable("flight_time_tbl")

In [0]:
%sql
describe extended flight_time_tbl

In [0]:
%fs ls /user/hive/warehouse/flight_time_tbl

In [0]:
%sql
show tables

In [0]:
spark.catalog.listTables()

In [0]:
%sql
drop table flight_time_tbl

In [0]:
%sql
create table if not exists ext_flight_time_tbl like flight_time_tbl
location "/user/hive/warehouse/flight_time_tbl"

In [0]:
%sql
describe extended ext_flight_time_tbl

In [0]:
spark.catalog.listTables()

In [0]:
%sql
select count(*) from ext_flight_time_tbl

In [0]:
%sql
drop table ext_flight_time_tbl

In [0]:
spark.catalog.listTables()

In [0]:
%sql
select count(*) from flight_time_tbl